In [11]:
import xarray as xr
#from xclim import sdba
#from xclim.core.calendar import convert_calendar
#import xclim.indices as xci
#import xclim.ensembles as xce
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import gcsfs
import zarr
import os


# lat and lon coordinates for Edmonton
#lat_edm = 
#lon_edm = 

# time periods for historical and future periods
years_hist = range(1980, 2011) # remember that range(start, end) is not inclusive of `end`
years_future = range(2030, 2061)

# url for the CSV file that contains the data catalog
url_gcsfs_catalog = 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv'

In [28]:
# Open the NetCDF file
file_path = 'G:/Other computers/My MacBook Air/Documents/School/MEnvSc/UTCDW Hackathon/TerraClimate.tmax.monthlymean.1980-2010.nc'  
nc_data = xr.open_dataset(file_path)

# Extract longitude and latitude values
stn_lon = nc_data.lon.values
stn_lat = nc_data.lat.values

# Create an xarray Dataset with lon and lat as coordinates
coords = {'lon': stn_lon, 'lat': stn_lat}
coords_ds = xr.Dataset(coords=coords)

# Convert the calendar if needed (optional)
#coords_ds_noleap = convert_calendar(coords_ds, 'noleap')

# Access the extracted variables if needed (optional)
#tas_obs_noleap = coords_ds_noleap.tas

# Display the coordinates dataset
print(coords_ds)


<xarray.Dataset>
Dimensions:  (lon: 60, lat: 120)
Coordinates:
  * lon      (lon) float64 32.02 32.06 32.1 32.15 ... 34.35 34.4 34.44 34.48
  * lat      (lat) float64 15.98 15.94 15.9 15.85 ... 11.15 11.1 11.06 11.02
Data variables:
    *empty*


In [29]:
# open the Google Cloud model data catalog with pandas
df_catalog = pd.read_csv(url_gcsfs_catalog)

# search for models which have daily tas data from the historical and SSP3-3.0 scenarios
search_string_mm = (
    "(table_id == 'day') & "
    "(variable_id in ['tasmax', 'tasmin', 'pr', 'hur']) & "
    "(member_id == 'r1i1p1f1') & "
    "(experiment_id in ['ssp245'])"
)
df_search_mm = df_catalog.query(search_string_mm).sort_values('source_id')
df_search_mm

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\indra\anaconda3\Lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\indra\anaconda3\Lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
           ^^^^^^^^
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\core\frame.py", line 1175, in _repr_html_
    else:
          
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
            ^^^^^^^^^^^^^
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\html.py", line 94, in render
    self._write_table()
  File "C:\Users

        activity_id institution_id      source_id experiment_id member_id  \
378916  ScenarioMIP   CSIRO-ARCCSS     ACCESS-CM2        ssp245  r1i1p1f1   
378917  ScenarioMIP   CSIRO-ARCCSS     ACCESS-CM2        ssp245  r1i1p1f1   
379701  ScenarioMIP   CSIRO-ARCCSS     ACCESS-CM2        ssp245  r1i1p1f1   
379704  ScenarioMIP   CSIRO-ARCCSS     ACCESS-CM2        ssp245  r1i1p1f1   
203687  ScenarioMIP            AWI  AWI-CM-1-1-MR        ssp245  r1i1p1f1   
...             ...            ...            ...           ...       ...   
380485  ScenarioMIP            NCC     NorESM2-MM        ssp245  r1i1p1f1   
380505  ScenarioMIP            NCC     NorESM2-MM        ssp245  r1i1p1f1   
515029  ScenarioMIP        AS-RCEC        TaiESM1        ssp245  r1i1p1f1   
515020  ScenarioMIP        AS-RCEC        TaiESM1        ssp245  r1i1p1f1   
515032  ScenarioMIP        AS-RCEC        TaiESM1        ssp245  r1i1p1f1   

       table_id variable_id grid_label  \
378916      day      tasmin      

In [30]:
# refine the list to include only the chosen models
models = ['AWI-CM-1-1-MR', 'CESM2', 'CanESM5', 'EC-Earth3', 
          'FGOALS-g3','INM-CM5-0', 'IPSL-CM6A-LR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0']
df_search_mm = df_search_mm.query(f"source_id == {models}")
df_search_mm

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\indra\anaconda3\Lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\indra\anaconda3\Lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
           ^^^^^^^^
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\core\frame.py", line 1175, in _repr_html_
    else:
          
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
            ^^^^^^^^^^^^^
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "C:\Users\indra\anaconda3\Lib\site-packages\pandas\io\formats\html.py", line 94, in render
    self._write_table()
  File "C:\Users

        activity_id       institution_id      source_id experiment_id  \
203687  ScenarioMIP                  AWI  AWI-CM-1-1-MR        ssp245   
203680  ScenarioMIP                  AWI  AWI-CM-1-1-MR        ssp245   
203625  ScenarioMIP                  AWI  AWI-CM-1-1-MR        ssp245   
203618  ScenarioMIP                  AWI  AWI-CM-1-1-MR        ssp245   
110222  ScenarioMIP                CCCma        CanESM5        ssp245   
110210  ScenarioMIP                CCCma        CanESM5        ssp245   
110221  ScenarioMIP                CCCma        CanESM5        ssp245   
110311  ScenarioMIP                CCCma        CanESM5        ssp245   
425694  ScenarioMIP  EC-Earth-Consortium      EC-Earth3        ssp245   
425877  ScenarioMIP  EC-Earth-Consortium      EC-Earth3        ssp245   
425687  ScenarioMIP  EC-Earth-Consortium      EC-Earth3        ssp245   
255053  ScenarioMIP                  CAS      FGOALS-g3        ssp245   
255060  ScenarioMIP                  CAS      FGOAL

In [38]:
# function for pre-processing data
def get_and_process_data(catalog_df, model, scenario, gcs, lat, lon, years):
    # get the ztore url for this model and scenario
    df_scen = df_catalog.query(f"source_id == '{model}' & experiment_id == '{scenario}'")
    zstore_url = df_scen.zstore.values[0]
    
    # get the GCS mapper from the url
    mapper = gcs.get_mapper(zstore_url)
    
    # open the file with xarray
    ds = xr.open_zarr(mapper, consolidated = True)
    
    # get the tas data, select the time period, and interp to the desired location
    tas_loc = ds.tas.sel(time = ds.time.dt.year.isin(years)).interp(lat = lat, lon = lon)
    
    # drop 'height' coordinate, which is always 2m but isn't present on all datasets
    if 'height' in tas_loc.coords.keys():
        tas_loc = tas_loc.reset_coords('height', drop = True)
        
    # some datasets put the date at 12:00 whereas some put it at 00:00. To make all
    # of them consistent, simply change the time coordinate to the date only
    tas_loc = tas_loc.assign_coords(time = tas_loc.time.dt.floor('D'))
    # convert from Kelvin to Celsius and return
    
    tas_loc = tas_loc - 273.15
    return tas_loc.compute()

In [39]:
# function for downloading the data
def download_data_multimodel(catalog_df, gcs, models, scenario,
                             stn_lat, stn_lon,
                             years_hist, years_future):

    ds_list_hist = []
    ds_list_future = []
    for model in models:
        print(f"========================{model}=============================")
        print('historical')
        tas_model_hist = get_and_process_data(catalog_df, model, 'historical', 
                                              gcs, stn_lat, stn_lon, years_hist)
        ds_list_hist.append(tas_model_hist)
    
        # get the future simulation data for this model and scenario
        print(scenario)
        tas_model_future = get_and_process_data(catalog_df, model, scenario, 
                                                gcs, stn_lat, stn_lon, years_future)
        ds_list_future.append(tas_model_future)

    print('finished acquiring model data')
    
    # concatenate the ds_lists together
    ds_ens_hist_raw = xce.create_ensemble(ds_list_hist,                             
                                          realizations = models,
                                          calendar = 'noleap') # common calendar to place all models onto

    ds_ens_future_raw = xce.create_ensemble(ds_list_future, 
                          # names of each model, which is represented by a dimension called 'realization'
                                           realizations = models,
                                           calendar = 'noleap')

    
    # return
    return ds_ens_hist_raw, ds_ens_future_raw

In [40]:
# authenticate access to Google Cloud
gcs = gcsfs.GCSFileSystem(token='anon')

# file names to save the downloaded data, to save time later when re-running this notebook
fout_hist = 'data_files/tasmax.cmip6.daily.historical.r4i1p1f1.1980-2010.EA.nc'
fout_future = 'data_files/tasmax.cmip6.daily.ssp3.r4i1p1f1.2030-2060.EA.nc'

# use the function to download the data, this may take a few minutes to run
if (not os.path.exists(fout_hist)) or (not os.path.exists(fout_future)):
    ds_ens_hist_raw, ds_ens_ssp2_raw = download_data_multimodel(df_search_mm, gcs, models, "ssp245",
                                                                  stn_lat, stn_lon, years_hist, years_future)

    # write the data to output files 
    ds_ens_hist_raw.to_netcdf(fout_hist)
    ds_ens_ssp3_raw.to_netcdf(fout_future)
else:
    # open the files that already exist
    ds_ens_hist_raw = xr.open_dataset(fout_hist)
    ds_ens_ssp3_raw = xr.open_dataset(fout_future)

========================AWI-CM-1-1-MR=============================
historical


AttributeError: module 'sparse' has no attribute 'SparseArray'

In [37]:
df_scen

NameError: name 'df_scen' is not defined